In [ ]:
import pandas as pd, numpy as np

In [ ]:
trainset = extract1.copy()
features = trainset.drop('attack', axis=1)
labels = trainset['attack']

predset = extract2.copy()
preds = predset.drop('attack', axis=1)
pred_label = predset['attack']

np.shape(features), np.shape(labels), np.shape(preds), np.shape(pred_label)

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedShuffleSplit, RandomizedSearchCV

def bestGBDTNextModel(model, isKfold, nfold, searchCV, Xtrain, ytrain, Xtest, ytest, nIter, scoring, errScore, verbose, nJobs):
    grd_prams = {}
    classifier = LGBMClassifier(random_state=0, objective='binary:logistic')
    cv = KFold(n_splits=nfold, shuffle=True, random_state=0)
    if model == 'LGBM':  
        grd_prams.update({'max_depth': [50, 100],
              'learning_rate' : [0.01, 0.05],
              'num_leaves': [150, 200],
              'n_estimators': [300, 400],
              'num_boost_round':[4000, 5000],
              'subsample': [0.5, 1],
              'reg_alpha': [0.01, 0.1],
              'reg_lambda': [0.01, 0.1],
              'min_data_in_leaf': [20, 30],
              'lambda_l1': [0.01, 0.1],
              'lambda_l2': [0.01, 0.1]
            })
        classifier = LGBMClassifier(random_state=0, boosting_type='gbdt', objective='binary', metric='auc')
    if isKfold == False:
        cv = StratifiedShuffleSplit(n_splits=nfold, test_size=0.2, random_state=0)
    grid_ = RandomizedSearchCV(classifier, param_distributions=grd_prams,
                               n_iter=nIter, scoring=scoring, error_score=errScore, verbose=verbose, n_jobs=nJobs, cv=cv)
    grid_.fit(Xtrain, ytrain)
    score_ = grid_.score(Xtest, ytest)
    print("{} grid_.best_score {}".format(model, np.round(grid_.best_score_,3)))
    print("{} grid_.best_score {}".format(model, np.round(score_,3)))
    print("{} best_estimator {}".format(model, grid_.best_estimator_))
    return grid_.best_params_

In [ ]:
best_param = bestGBDTNextModel('LGBM', False, 5, 'RANDOM', features, labels, preds, pred_label, 15, 'roc_auc', 0, 3, -1)
lgbm = LGBMClassifier(**best_param)
score_lgbm = lgbm.fit(features, labels).score(preds, pred_label)
print("score_lgbm ::: {}".format(score_lgbm))
print("-----------------------------------")
y_lgbm = lgbm.predict(preds)
print(classification_report(pred_label, y_lgbm))

In [ ]:
lgbm = LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', lambda_l1=0.01, lambda_l2=0.1,
               learning_rate=0.05, max_depth=50, metric='auc',
               min_child_samples=20, min_child_weight=0.001,
               min_data_in_leaf=30, min_split_gain=0.0, n_estimators=300,
               n_jobs=-1, num_boost_round=5000, num_leaves=150,
               objective='binary', random_state=0, reg_alpha=0.1,
               reg_lambda=0.01, silent=True, subsample=0.5,
               subsample_for_bin=200000, subsample_freq=0)
score = lgbm.fit(X_resampled, y_resampled).score(preds, pred_label)
print("score_lgbm ::: {}".format(score))
print("-----------------------------------")
y_lgbm = lgbm.predict(preds)
print(classification_report(pred_label, y_lgbm))


In [ ]:
trainset = testset1.copy()
features = trainset.drop(['attack','attack_P1','attack_P2','attack_P3'], axis=1)
labels = trainset['attack']

features = np.array(features)
labels = np.array(labels)

predset = testset2.copy()
preds = predset.drop(['attack','attack_P1','attack_P2','attack_P3'], axis=1)
pred_label = predset['attack']

preds = np.array(preds)
pred_label = np.array(pred_label)

In [ ]:
np.shape(features), np.shape(labels), np.shape(preds), np.shape(pred_label)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredLogarithmicError
from tensorflow.keras.losses import MeanSquaredLogarithmicError, MeanSquaredError
from tensorflow.keras.layers import Dense, LSTM, GRU, Bidirectional, RNN, BatchNormalization, TimeDistributed, LeakyReLU
from tensorflow.keras.experimental import PeepholeLSTMCell
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1, l2, l1_l2

In [ ]:
def build_model():
    model = Sequential()
    model.add(layers.Embedding(3600, 60, input_length=59))
    model.add(layers.Dropout(.3))
    model.add(layers.Conv1D(60, 3, padding='valid', activation='relu'))
    
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model
model = build_model()
model.summary()

In [ ]:
# 로그 초기화
!rm -rf ./logs/RNN_n_1_LSTM/

# 척도
#metrics = tf.keras.metrics.BinaryAccuracy()
metrics = tf.keras.metrics.Accuracy()

# 손실함수
#loss = tf.keras.losses.CosineSimilarity()
#loss = tf.keras.losses.CategoricalCrossentropy()
loss = tf.keras.losses.BinaryCrossentropy()

# 최적화기
#optimizer = tf.keras.optimizers.Adadelta(lr=0.1)  # 종종걸음 너무 작아져서 정지하는걸 막아보자.
#optimizer = tf.keras.optimizers.SGD(lr=0.1)  # 전부 다봐야 하는데 한걸음은 너무 오래 걸리니까 조금만 보고 빨리 판단한다. 같은시간에 더 많이 간다.
#optimizer = tf.keras.optimizers.RMSprop(lr=0.001)  # 보폭을 줄이는건 좋은데 이전 맥락 상황봐가며 하자.
optimizer = tf.keras.optimizers.Adam(lr=0.001)  # RMSProp + Momentum 방향도 스텝사이즈도 적절하게!(Momentum:스텝 계산해서 움직인 후, 아까 내려오던 관성방향 또 가자)
#optimizer = tf.keras.optimizers.Adagrad(lr=0.1)  # 안가본곳은 성큼 빠르게 훑고, 많이 가본곳은 잘아니까 갈수록 보폭을 줄여 세밀히 탐색
#optimizer = tf.keras.optimizers.Adamax(lr=0.1)
#optimizer = tf.keras.optimizers.Nadam(lr=0.01)  # Adam에 Momentum 대신에 NAG를 붙이자(NAG:일단 관성방향 먼저 움직이고, 움직인 자리에 스텝을 계산하니 더 빠르더라)
#optimizer = tf.keras.optimizers.Ftrl(lr=0.01)

## 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 3600

# 훈련
data_dir = './model/CNN/'
logs_dir = './logs/CNN'
best_model_file = os.path.join(data_dir, 'best_model.h5')
checkpoint = tf.keras.callbacks.ModelCheckpoint(best_model_file, monitor='accuracy', save_weights_only=True, save_best_only=True)
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=logs_dir)
#early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, mode='min', restore_best_weights=True)
CALLBACKS = [checkpoint, tensorboard]
history = model.fit(features, labels,
                    epochs=NUM_EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=CALLBACKS,
                    validation_data=[preds, pred_label])

In [ ]:
history = model.evaluate(preds,pred_label, batch_size=BATCH_SIZE)
print("\nTest score:",history[0])
print("Test Accuracy:",history[1])

In [ ]:
best_model = build_model()
best_model.load_weights('model/CNN/best_model.h5')
best_model.compile(loss='binary_corssentropy', optimizer='adam', metrics=['accuracy'])

predictions_1 = model.predict(preds)
predictions_2 = best_model.predict(preds)

In [ ]:
for i in range(len(pred_label)):
    print('label : {}, prediction : {}'.format(pred_label[i], predictions_1[i]))